In [16]:
import sys 
import os
import numpy as np 
import random

In [17]:
def readFile(n, fileName):
    file = []
    with open (f"config/{fileName}_{n}.txt") as data:
        for row in data:
            file.append(row.replace('\n', ''))
        return file

In [18]:
def add_to_jobs(jobList, job):
    jobList.append(job)
    jobList.sort(key=lambda x: x.eventTime)

In [19]:
def queue_to_server(queue, jobList, masterclock):
    nextJob = queue.pop(0)
    nextJob.eventTime = masterclock
    add_to_jobs(jobList, nextJob)
    # print("server has a space, move from queue to server")

In [20]:
def log(job, masterclock):
    print("Master clock:", masterclock)
    print("Job:", job.index)
    print("Job group:", job.group)
    print("Job type:", job.type)
    print("Arrival Time", job.arrivalTime)
    print("Service Time:", job.serviceTime)

In [21]:
def mrt(jobs):
    response_times = {"0":0,"1":0}
    n = {"0":0, "1":0}
    mean = {"0":0, "1":0}
    for arr_time, dep_time, group in jobs:
        if group != "r0":
            mean[group] += (dep_time - arr_time)
            n[group] += 1
            
    response_times["0"] = mean["0"] / n["0"]
    response_times["1"] = mean["1"] / n["1"]

    return response_times

In [22]:
def output(s, completed_jobs, response_times):
    with open(f"output/mrt_{s}.txt", "w") as mrt_file:
                mrt_file.write(f"{mrt(completed_jobs)["0"]} "+ f"{mrt(completed_jobs)["1"]}")

    # Save dep values to a file
    with open(f"output/dep_{s}.txt", "w") as dep_file:
        for l in completed_jobs:
            dep_file.write(" ".join(map(str, l)) + "\n")

In [23]:
class job:
    def __init__(self, arrivalTime, serviceTime, group, index):
        self.arrivalTime = arrivalTime
        self.serviceTime = float(serviceTime)
        self.group = group
        self.departureTime = float(arrivalTime) + float(serviceTime)
        self.eventTime = arrivalTime
        self.index = index
        self.type = "Arrival"

In [24]:
def generate_groups(p0, n):
    jobsGroup = []
    for _ in range(n):
        random_number = random.random()
        if random_number < p0:
            jobsGroup.append("0")
        else:
            jobsGroup.append("1")
    return jobsGroup

In [25]:
def generate_arrivals(Lamba, alpha_2l, alpha_2u, endTime):
    interArrivalTime = []

    a1k = np.random.exponential(1 / Lamba)
    a2k = np.random.uniform(alpha_2l, alpha_2u)
    while True:
        if not interArrivalTime:
            ak = a1k * a2k
        else:
            ak = interArrivalTime[-1] + (a1k * a2k)
        if ak > endTime:
            break
        interArrivalTime.append(ak)
    return interArrivalTime

In [26]:
def inverse_pdf0(g, alpha, beta, eta):
    gamma = alpha**(-eta) - beta**(-eta)
    return (eta/(gamma * g)) ** (1/(eta+1))

def inverse_pdf1(g, alpha, eta):
    gamma = alpha**(-eta)
    return (eta/(gamma * g)) ** (1/(eta+1))

In [27]:
def generate_services(alpha0, beta0, eta0, alpha1, eta1, interArrivalTime, jobsGroup):
    serviceTime = []
    rand_values = np.random.uniform(size=len(interArrivalTime))
    for i in range(len(interArrivalTime)):
        if jobsGroup[i] == "0":
            serviceTime.append(inverse_pdf0(rand_values[i], alpha0, beta0, eta0))
        else:
            serviceTime.append(inverse_pdf1(rand_values[i], alpha1, eta1))

    return serviceTime

In [28]:
def get_all_jobs(mode, interArrivalFile, serviceFile, paraFile):
    arrivalTime = []
    serviceTime = []
    jobsGroup = []

    jobList = []

    if mode == "trace":
        interArrivalTime = list(float(i) for i in interArrivalFile)

        for line in serviceFile:
            serviceTime.append(line.split(" ")[0])
            jobsGroup.append(line.split(" ")[1])

        arrivalTime = np.cumsum(interArrivalTime)
        
        for i in range(len(interArrivalFile)):
            jobList.append(job(arrivalTime[i], serviceTime[i], jobsGroup[i], index=i))

    elif mode == "random":
        endTime = int(paraFile[3])
        Lamba, alpha_2l, alpha_2u = tuple(float(e) for e in interArrivalFile[0].split())
        p0 = float(serviceFile[0])
        alpha0, beta0, eta0  = tuple(float(e) for e in serviceFile[1].split())
        alpha1, eta1  = tuple(float(e) for e in serviceFile[2].split())

        arrivalTime = generate_arrivals(Lamba, alpha_2l, alpha_2u, endTime)
        jobsGroup = generate_groups(p0, len(arrivalTime))
        serviceTime = generate_services(alpha0, beta0, eta0, alpha1, eta1, arrivalTime, jobsGroup)
        
        for i in range(len(arrivalTime)):
            jobList.append(job(arrivalTime[i], serviceTime[i], jobsGroup[i], index=i))
        
        
    return jobList
        
    

In [29]:
def main(s):
    # Read all files:
    interArrivalFile = readFile(s, 'interarrival')
    mode = readFile(s, 'mode')[0]
    paraFile = readFile(s, 'para')
    serviceFile = readFile(s, 'service')

    n = int(paraFile[0])
    n0 = int(paraFile[1])
    Tlimit = float(paraFile[2])

    masterclock = 0
    group0 = 0
    group1 = 0
    queue0 = []
    queue1 = []

    completed_jobs = []
    response_times = {"0":0,"1":0}

    jobList = get_all_jobs(mode, interArrivalFile, serviceFile, paraFile)
    
    while jobList:
        currentJob = jobList.pop(0)
        masterclock = currentJob.eventTime
        if currentJob.type == "Arrival":
            # log(currentJob, masterclock)
            if currentJob.group == "0":
                if group0 != n0:
                    # print("Group 0 has a space.")
                    group0 += 1
                    if currentJob.serviceTime > Tlimit:
                        currentJob.departureTime = masterclock + Tlimit
                        currentJob.eventTime = currentJob.departureTime
                        currentJob.group = "R"
                        currentJob.type = "Departure"
                        add_to_jobs(jobList, currentJob)
                        # print("Job", f"{currentJob.index}", "cannot finish, need to go to group1 later.")
                    else:
                        currentJob.departureTime = masterclock + currentJob.serviceTime
                        currentJob.eventTime = currentJob.departureTime
                        currentJob.type = "Departure"
                        add_to_jobs(jobList,currentJob)
                        # print("Job", f"{currentJob.index}", "will finish at", f"{currentJob.departureTime}")
                else:
                    print("Group 0 is full")
                    queue0.append(currentJob)
                    # print("Job",f"{currentJob.index}","to queue0: ", currentJob.eventTime)
            elif currentJob.group == "1" or currentJob.group == "r0":
                if group1 != n-n0:
                    # print("Group 1 has a space")
                    group1 += 1
                    currentJob.departureTime = masterclock + currentJob.serviceTime
                    currentJob.eventTime = currentJob.departureTime
                    currentJob.type = "Departure"
                    add_to_jobs(jobList, currentJob)
                    # print("Job", f"{currentJob.index}", "will finish at", f"{currentJob.departureTime}")
                else:
                    # print("Group 1 is full")
                    queue1.append(currentJob)
                    # print("Job",f"{currentJob.index}","to queue1: ", currentJob.eventTime)
        elif currentJob.type ==  "Departure":
            # log(currentJob, masterclock)
            if currentJob.group == "0":
                # print("Job", f"{currentJob.index}", "finished.")
                completed_jobs.append((currentJob.arrivalTime, masterclock, currentJob.group))
                if queue0:
                    queue_to_server(queue0, jobList, masterclock)
                group0 -= 1
            elif currentJob.group == "R":
                currentJob.type = "Arrival"
                currentJob.departureTime = masterclock + currentJob.serviceTime
                currentJob.eventTime = masterclock
                currentJob.group = "r0"
                add_to_jobs(jobList, currentJob)
                # print("Job", f"{currentJob.index}", "has moved to group 1")
                if queue0:
                    queue_to_server(queue0, jobList, masterclock)
                group0 -= 1
            else:
                # print("Job", f"{currentJob.index}", "finished.")
                completed_jobs.append((currentJob.arrivalTime, masterclock, currentJob.group))
                if queue1:
                    queue_to_server(queue1, jobList, masterclock)
                group1 -= 1
        # print("-----------------------------")
    # print(completed_jobs)
    # print(mrt(completed_jobs)["0"], mrt(completed_jobs)["1"])
    output(s, completed_jobs, response_times) 
        
    

In [30]:
main(4)

[2.0548651661904644, 4.109730332380929, 6.164595498571393, 8.219460664761858, 10.274325830952321, 12.329190997142785, 14.384056163333248, 16.438921329523712, 18.493786495714176, 20.54865166190464, 22.603516828095103, 24.658381994285566, 26.71324716047603, 28.768112326666493, 30.822977492856957, 32.877842659047424, 34.93270782523789, 36.98757299142836, 39.042438157618825, 41.09730332380929, 43.15216848999976, 45.20703365619023, 47.261898822380694, 49.31676398857116, 51.37162915476163, 53.426494320952095, 55.48135948714256, 57.53622465333303, 59.591089819523496, 61.64595498571396, 63.70082015190443, 65.75568531809489, 67.81055048428536, 69.86541565047582, 71.92028081666629, 73.97514598285676, 76.03001114904723, 78.0848763152377, 80.13974148142816, 82.19460664761863, 84.2494718138091, 86.30433697999956, 88.35920214619003, 90.4140673123805, 92.46893247857096, 94.52379764476143, 96.5786628109519, 98.63352797714236, 100.68839314333283, 102.7432583095233, 104.79812347571377, 106.8529886419042